In [30]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping

In [52]:
TIMEFRAMES = [7, 14, 30, 90, 180]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

TRANSFER = False
if TRANSFER:
    labels = pd.read_csv('labels/transfer_labels.csv')
else:
    labels = pd.read_csv('labels/labels.csv')

In [33]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/Line/AI&Robotics_2019-02-01 00:00:00_7.png',
    'images/Line/AI&Robotics_2019-02-01 00:00:00_14.png',
    'images/Line/AI&Robotics_2019-02-01 00:00:00_30.png',
    'images/Line/AI&Robotics_2019-02-01 00:00:00_90.png',
    'images/Line/AI&Robotics_2019-02-01 00:00:00_180.png',   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

80 105 38 115
80 100 85 120
80 100 132 120
80 100 226 120
80 100 414 120


In [34]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 7:
            # Crop the image using the bounding rectangle
            crop = img[105:105+115, 80:80+38]
        elif number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [44]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_35673/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [45]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [46]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                  Image  TimePrediction  \
0     images/OHLC/S&P500_2019-02-01 00:00:00_7_cropp...               5   
7844  images/AlgoTrading/Treasury_2019-02-01 00:00:0...              30   
5094  images/Line/EmergingMarkets_2019-02-01 00:00:0...               5   
5124  images/Line/EmergingMarkets_2019-02-01 00:00:0...               5   
910   images/OHLC/DAX_2019-02-01 00:00:00_30_cropped...              30   

       LastPrice  FuturePrice  Label  \
0     270.059998   270.140015      1   
7844  120.959999   119.589996      0   
5094   42.759998    42.400002      0   
5124   42.759998    42.400002      0   
910    25.042000    25.700001      1   

                                            Image_Array       Date  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
7844  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
5094  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
5124  [[0, 0, 0, 0, 0, 0, 0, 0, 

In [47]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    4744
0    3180
Name: Label, dtype: int64


In [ ]:
'''def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Set dilation rate and strides
        strides = (1, 3)

        # Add Convolutional layer
        model.add(Conv2D(filters, kernel_size=(5, 3), strides=strides, padding='same',
                         input_shape=input_shape if i == 0 else model.output_shape[1:]))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer only if dimensions allow
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax')) 

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model'''

In [48]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    # Determine the number of blocks based on the input image size
    if num_blocks == 2:
        dilation_rate = (1, 1)
    elif num_blocks == 3:
        dilation_rate = (1, 2)
    else:
        dilation_rate = (1, 3)

    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Add Convolutional layer
        if i == 0:  # Apply specific strides and dilation rate only for the first layer
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 3), padding='same', 
                             input_shape=input_shape if i == 0 else None))
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 1), dilation_rate=dilation_rate, padding='same', 
                             input_shape=input_shape if i == 0 else None))
        else:
            model.add(Conv2D(filters, kernel_size=(5, 3), padding='same'))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer
        model.add(MaxPooling2D(pool_size=(2, 1)))

    # Add Flatten and Dense layers for final prediction
    #dense_units = {2: 64, 3: 128, 4: 264}.get(num_blocks, 64) 
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

    return model

In [49]:
def get_investment_return(y_pred_binary, lastPrice, futurePrice):
    # Calculate Rate of Return RoR for each trade independently
    # Long positions
    long_ror = []
    long_trades = []
    long_investment = 0
    # Short positions
    short_ror = []
    short_trades = []
    short_investment = 0

    y_pred_binary = y_pred_binary.tolist()
    for i in range(len(y_pred_binary)):
        # Long position
        if y_pred_binary[i][0] == 1:
            long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
            long_ror.append(long_return_i)
            long_trades.append(100*(long_return_i))
            long_investment += 100
        # Short position
        else:
            short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
            short_ror.append(short_return_i)
            short_trades.append(100*(short_return_i))
            short_investment += 100
    
    if long_investment > 0:
        # Calculate average long RoR
        long_avg_ror = np.mean(long_ror)
        # Sum all long trades
        long_trades = np.sum(long_trades)
    else:
        long_avg_ror = 0
        long_trades = 0
    
    if short_investment > 0:
        # Calculate average short RoR
        short_avg_ror = np.mean(short_ror)
        # Sum all short trades
        short_trades = np.sum(short_trades)
    else:
        short_avg_ror = 0
        short_trades = 0

    return long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment

In [ ]:
# Train and evaluate the model for each img_type, timeframe, and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Initialize variables for weighted sums and total count
                weighted_accuracy_sum = 0
                weighted_precision_sum = 0
                weighted_recall_sum = 0
                weighted_f1_score_sum = 0
                weighted_hit_rate_sum = 0
                total_predictions = 0
                long_avg_ror = []
                long_trades = []
                long_investment = []
                short_avg_ror = []
                short_trades = []
                short_investment = []
                total_investment = []
                total_returns = []
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 7:
                    model = create_cnn_model(2, input_shape=(115, 38, 1))
                elif timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(3, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(4, input_shape=(120, 414, 1))
                
                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Split the data into a larger initial training set and a final test set
                split_index = int(len(X) * 0.8)
                X_train_full, X_test = X[:split_index], X[split_index:]
                y_train_full, y_test = y[:split_index], y[split_index:]
                lastPrice_train_full, lastPrice_test = lastPrice[:split_index], lastPrice[split_index:]
                futurePrice_train_full, futurePrice_test = futurePrice[:split_index], futurePrice[split_index:]

                class_weights = compute_class_weight('balanced', classes=np.unique(y_train_full), y=y_train_full)
                class_weight_dict = dict(enumerate(class_weights))

                # Define the rolling window size for training within the initial training set
                window_size = len(X_train_full) / 10

                # Early stopping callback
                early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

                for window in range(5):
                    start = 0
                    if window == 0:
                        end = int(len(X_train_full) * 0.4)
                    else:
                        start = end
                        end = start + int(window_size)

                    # Buffer = window size
                    # Define end of buffer
                    buffer_end = end + int(window_size)

                    # Ensure buffer_end does not exceed the length of the initial training set
                    buffer_end = min(buffer_end, len(X_train_full))

                    # Split the data into training and test sets in temporal order
                    X_train_temp = X_train_full[start:end]
                    y_train_temp = y_train_full[start:end]
                    lastPrice_train_temp = lastPrice_train_full[start:end]
                    futurePrice_train_temp = futurePrice_train_full[start:end]

                    X_test_temp = X_train_full[buffer_end:] if window == 4 else X_train_full[buffer_end:buffer_end+int(window_size*2)]
                    y_test_temp = y_train_full[buffer_end:] if window == 4 else y_train_full[buffer_end:buffer_end+int(window_size*2)]
                    lastPrice_test_temp = lastPrice_train_full[buffer_end:] if window == 4 else lastPrice_train_full[buffer_end:buffer_end+int(window_size*2)]
                    futurePrice_test_temp = futurePrice_train_full[buffer_end:] if window == 4 else futurePrice_train_full[buffer_end:buffer_end+int(window_size*2)]

                    model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2, class_weight=class_weight_dict)   
                
                    # Evaluate the model
                    y_pred = model.predict(X_test_temp)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary = np.where(y_pred > 0.55, 1, 0)

                    num_predictions = len(y_test_temp)
                    
                    # Calculate the performance metrics for the current epoch
                    current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                    current_precision = precision_score(y_test_temp, y_pred_binary)
                    current_recall = recall_score(y_test_temp, y_pred_binary)
                    current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                    y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                    correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                    current_hit_rate = (correct_predictions / num_predictions)

                    # Update the weighted sums and total count
                    weighted_accuracy_sum += current_accuracy * num_predictions
                    weighted_precision_sum += current_precision * num_predictions
                    weighted_recall_sum += current_recall * num_predictions
                    weighted_f1_score_sum += current_f1_score * num_predictions
                    weighted_hit_rate_sum += current_hit_rate * num_predictions
                    total_predictions += num_predictions

                    long_avg_ror_temp, long_trades_temp, long_investment_temp, short_avg_ror_temp, short_trades_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                    # Total investment
                    long_avg_ror.append(long_avg_ror_temp)
                    long_trades.append(long_trades_temp)
                    long_investment.append(long_investment_temp)
                    short_avg_ror.append(short_avg_ror_temp)
                    short_trades.append(short_trades_temp)
                    short_investment.append(short_investment_temp)
                
                # replace nan values with 0
                long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
                long_trades = [0 if np.isnan(x) else x for x in long_trades]
                long_investment = [0 if np.isnan(x) else x for x in long_investment]
                short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
                short_trades = [0 if np.isnan(x) else x for x in short_trades]
                short_investment = [0 if np.isnan(x) else x for x in short_investment]

                # Calculate the weighted averages after the epoch loop
                weighted_average_accuracy = weighted_accuracy_sum / total_predictions
                weighted_average_precision = weighted_precision_sum / total_predictions
                weighted_average_recall = weighted_recall_sum / total_predictions
                weighted_average_f1_score = weighted_f1_score_sum / total_predictions
                weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                            
                print("Evaluation Metrics:")
                print(f"Accuracy: {weighted_average_accuracy}")
                print(f"Precision: {weighted_average_precision}")
                print(f"Recall: {weighted_average_recall}")
                print(f"F1 Score: {weighted_average_f1_score}")
                print(f"Hit Rate: {weighted_average_hit_rate}")
                print(f"Average Long RoR: {np.mean(long_avg_ror)}")
                print(f"Long Investment: {np.mean(long_investment)}")
                print(f"Long Return: {np.mean(long_trades)}")
                print(f"Average Short RoR: {np.mean(short_avg_ror)}")
                print(f"Short Investment: {np.mean(short_investment)}")
                print(f"Short Return: {np.mean(short_trades)}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': weighted_average_accuracy,
                    'Precision': weighted_average_precision,
                    'Recall': weighted_average_recall,
                    'F1_Score': weighted_average_f1_score,
                    'Hit_Rate': weighted_average_hit_rate,
                    'Long_Average_RoR': np.mean(long_avg_ror),
                    'Long_Investment': np.mean(long_investment),
                    'Long_Return': np.mean(long_trades),
                    'Short_Average_RoR': np.mean(short_avg_ror),
                    'Short_Investment': np.mean(short_investment),
                    'Short_Return': np.mean(short_trades)
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation/separate/evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation/separate/evaluation_scores.csv'.")

In [51]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction <= timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")

            # Initialize variables for weighted sums and total count
            weighted_accuracy_sum = 0
            weighted_precision_sum = 0
            weighted_recall_sum = 0
            weighted_f1_score_sum = 0
            weighted_hit_rate_sum = 0
            total_predictions = 0
            long_avg_ror = []
            long_trades = []
            long_investment = []
            short_avg_ror = []
            short_trades = []
            short_investment = []
            total_investment = []
            total_returns = []
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 7:
                model = create_cnn_model(2, input_shape=(115, 38, 1))
            elif timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(3, input_shape=(120, 132, 1))
            elif timeframe == 90:
                model = create_cnn_model(3, input_shape=(120, 226, 1))
            elif timeframe == 180:
                model = create_cnn_model(4, input_shape=(120, 414, 1))
            
            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = data['LastPrice'].tolist()
            futurePrice = data['FuturePrice'].tolist()
            y = data['Label'].values

            # Split the data into a larger initial training set and a final test set
            split_index = int(len(X) * 0.8)
            X_train_full, X_test = X[:split_index], X[split_index:]
            y_train_full, y_test = y[:split_index], y[split_index:]
            lastPrice_train_full, lastPrice_test = lastPrice[:split_index], lastPrice[split_index:]
            futurePrice_train_full, futurePrice_test = futurePrice[:split_index], futurePrice[split_index:]

            class_weights = compute_class_weight('balanced', classes=np.unique(y_train_full), y=y_train_full)
            class_weight_dict = dict(enumerate(class_weights))

            # Define the rolling window size for training within the initial training set
            window_size = len(X_train_full) / 10

            # Early stopping callback
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            for window in range(5):
                start = 0
                if window == 0:
                    end = int(len(X_train_full) * 0.4)
                else:
                    start = end
                    end = start + int(window_size)

                # Buffer = window size
                # Define end of buffer
                buffer_end = end + int(window_size)

                # Ensure buffer_end does not exceed the length of the initial training set
                buffer_end = min(buffer_end, len(X_train_full))

                # Split the data into training and test sets in temporal order
                X_train_temp = X_train_full[start:end]
                y_train_temp = y_train_full[start:end]
                lastPrice_train_temp = lastPrice_train_full[start:end]
                futurePrice_train_temp = futurePrice_train_full[start:end]

                X_test_temp = X_train_full[buffer_end:] if window == 4 else X_train_full[buffer_end:buffer_end+int(window_size*2)]
                y_test_temp = y_train_full[buffer_end:] if window == 4 else y_train_full[buffer_end:buffer_end+int(window_size*2)]
                lastPrice_test_temp = lastPrice_train_full[buffer_end:] if window == 4 else lastPrice_train_full[buffer_end:buffer_end+int(window_size*2)]
                futurePrice_test_temp = futurePrice_train_full[buffer_end:] if window == 4 else futurePrice_train_full[buffer_end:buffer_end+int(window_size*2)]

                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2, class_weight=class_weight_dict)   
            
                # Evaluate the model
                y_pred = model.predict(X_test_temp)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.55, 1, 0)

                num_predictions = len(y_test_temp)
                
                # Calculate the performance metrics for the current epoch
                current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                current_precision = precision_score(y_test_temp, y_pred_binary)
                current_recall = recall_score(y_test_temp, y_pred_binary)
                current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                current_hit_rate = (correct_predictions / num_predictions)

                # Update the weighted sums and total count
                weighted_accuracy_sum += current_accuracy * num_predictions
                weighted_precision_sum += current_precision * num_predictions
                weighted_recall_sum += current_recall * num_predictions
                weighted_f1_score_sum += current_f1_score * num_predictions
                weighted_hit_rate_sum += current_hit_rate * num_predictions
                total_predictions += num_predictions

                long_avg_ror_temp, long_trades_temp, long_investment_temp, short_avg_ror_temp, short_trades_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                # Total investment
                long_avg_ror.append(long_avg_ror_temp)
                long_trades.append(long_trades_temp)
                long_investment.append(long_investment_temp)
                short_avg_ror.append(short_avg_ror_temp)
                short_trades.append(short_trades_temp)
                short_investment.append(short_investment_temp)
            
            # replace nan values with 0
            long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
            long_trades = [0 if np.isnan(x) else x for x in long_trades]
            long_investment = [0 if np.isnan(x) else x for x in long_investment]
            short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
            short_trades = [0 if np.isnan(x) else x for x in short_trades]
            short_investment = [0 if np.isnan(x) else x for x in short_investment]

            # Calculate the weighted averages after the epoch loop
            weighted_average_accuracy = weighted_accuracy_sum / total_predictions
            weighted_average_precision = weighted_precision_sum / total_predictions
            weighted_average_recall = weighted_recall_sum / total_predictions
            weighted_average_f1_score = weighted_f1_score_sum / total_predictions
            weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                        
            print("Evaluation Metrics:")
            print(f"Accuracy: {weighted_average_accuracy}")
            print(f"Precision: {weighted_average_precision}")
            print(f"Recall: {weighted_average_recall}")
            print(f"F1 Score: {weighted_average_f1_score}")
            print(f"Hit Rate: {weighted_average_hit_rate}")
            print(f"Average Long RoR: {np.mean(long_avg_ror)}")
            print(f"Long Investment: {np.mean(long_investment)}")
            print(f"Long Return: {np.mean(long_trades)}")
            print(f"Average Short RoR: {np.mean(short_avg_ror)}")
            print(f"Short Investment: {np.mean(short_investment)}")
            print(f"Short Return: {np.mean(short_trades)}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': weighted_average_accuracy,
                'Precision': weighted_average_precision,
                'Recall': weighted_average_recall,
                'F1_Score': weighted_average_f1_score,
                'Hit_Rate': weighted_average_hit_rate,
                'Long_Average_RoR': np.mean(long_avg_ror),
                'Long_Investment': np.mean(long_investment),
                'Long_Return': np.mean(long_trades),
                'Short_Average_RoR': np.mean(short_avg_ror),
                'Short_Investment': np.mean(short_investment),
                'Short_Return': np.mean(short_trades)
            }, ignore_index=True)

            # Save the model
            model.save(f"models/small_combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation/combined/small_evaluation_scores_combined.csv', index=False)
print("Evaluation scores saved to 'evaluation/combined/combined_model_evaluation_scores.csv'.")

Predicting 5 days ahead with 7 days timeframe.
Epoch 1/10
7/7 [==============================] - 4s 305ms/step - loss: 0.7449 - accuracy: 0.5047 - val_loss: 0.6390 - val_accuracy: 0.7037
Epoch 2/10
7/7 [==============================] - 2s 256ms/step - loss: 0.5502 - accuracy: 0.7150 - val_loss: 0.6535 - val_accuracy: 0.6111
Epoch 3/10
7/7 [==============================] - 2s 257ms/step - loss: 0.4439 - accuracy: 0.8131 - val_loss: 0.4454 - val_accuracy: 0.8148
Epoch 4/10
7/7 [==============================] - 2s 276ms/step - loss: 0.3535 - accuracy: 0.8178 - val_loss: 0.5332 - val_accuracy: 0.7778
Epoch 5/10
7/7 [==============================] - 2s 264ms/step - loss: 0.2453 - accuracy: 0.8972 - val_loss: 0.5425 - val_accuracy: 0.8148
Epoch 6/10
7/7 [==============================] - 2s 252ms/step - loss: 0.2622 - accuracy: 0.8645 - val_loss: 0.5634 - val_accuracy: 0.7963
Epoch 7/10
7/7 [==============================] - 2s 279ms/step - loss: 0.1609 - accuracy: 0.9439 - val_loss: 0.5

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
7/7 [==============================] - 6s 644ms/step - loss: 0.6424 - accuracy: 0.5794 - val_loss: 0.6295 - val_accuracy: 0.6481
Epoch 2/10
7/7 [==============================] - 4s 571ms/step - loss: 0.4097 - accuracy: 0.8271 - val_loss: 0.6088 - val_accuracy: 0.7407
Epoch 3/10
7/7 [==============================] - 4s 572ms/step - loss: 0.2081 - accuracy: 0.9346 - val_loss: 1.3075 - val_accuracy: 0.6852
Epoch 4/10
7/7 [==============================] - 4s 632ms/step - loss: 0.1907 - accuracy: 0.9206 - val_loss: 1.4882 - val_accuracy: 0.6852
Epoch 5/10
7/7 [==============================] - 4s 609ms/step - loss: 0.1126 - accuracy: 0.9720 - val_loss: 1.6627 - val_accuracy: 0.6667
Epoch 6/10
7/7 [==============================] - 4s 642ms/step - loss: 0.0952 - accuracy: 0.9626 - val_loss: 2.8807 - val_accuracy: 0.7037
Epoch 7/10
7/7 [==============================] - 4s 606ms/step - loss: 0.0419 - accuracy: 0.9953 - val_loss: 2.9070 - val_accuracy: 0.7037
Epoch 8/10
7/7 [====

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
7/7 [==============================] - 12s 2s/step - loss: 0.8126 - accuracy: 0.4976 - val_loss: 0.6657 - val_accuracy: 0.6731
Epoch 2/10
7/7 [==============================] - 11s 2s/step - loss: 0.6981 - accuracy: 0.5411 - val_loss: 0.7229 - val_accuracy: 0.3269
Epoch 3/10
7/7 [==============================] - 11s 2s/step - loss: 0.6874 - accuracy: 0.4879 - val_loss: 0.7474 - val_accuracy: 0.3269
Epoch 4/10
7/7 [==============================] - 11s 2s/step - loss: 0.6534 - accuracy: 0.5797 - val_loss: 0.7699 - val_accuracy: 0.5192
Epoch 5/10
7/7 [==============================] - 11s 2s/step - loss: 0.6003 - accuracy: 0.6957 - val_loss: 0.5587 - val_accuracy: 0.6538
Epoch 6/10
7/7 [==============================] - 11s 2s/step - loss: 0.5909 - accuracy: 0.6763 - val_loss: 0.6586 - val_accuracy: 0.6346
Epoch 7/10
7/7 [==============================] - 11s 2s/step - loss: 0.4837 - accuracy: 0.7729 - val_loss: 0.5511 - val_accuracy: 0.7308
Epoch 8/10
7/7 [==================

...

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as combined_180_30.h5
--------------------------------------------------
Predicting 90 days ahead with 180 days timeframe.
Epoch 1/10
6/6 [==============================] - 56s 9s/step - loss: 0.6167 - accuracy: 0.6719 - val_loss: 0.2234 - val_accuracy: 0.8776
Epoch 2/10
6/6 [==============================] - 58s 10s/step - loss: 0.3269 - accuracy: 0.8490 - val_loss: 0.2578 - val_accuracy: 0.8980
Epoch 3/10
6/6 [==============================] - 61s 11s/step - loss: 0.2593 - accuracy: 0.9062 - val_loss: 0.2444 - val_accuracy: 0.8980
Epoch 4/10
6/6 [==============================] - 56s 9s/step - loss: 0.1326 - accuracy: 0.9479 - val_loss: 0.2608 - val_accuracy: 0.8980
Epoch 5/10
6/6 [==============================] - 52s 9s/step - loss: 0.1466 - accuracy: 0.9479 - val_loss: 0.3345 - val_accuracy: 0.8980
Epoch 6/10
6/6 [==============================] - 53s 9s/step - loss: 0.1190 - accuracy: 0.9635 - val_loss: 0.3047 - val_accuracy: 0.8980
Epoch 7/10
6/6 [===================

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as combined_180_90.h5
--------------------------------------------------
Predicting 5 days ahead with 365 days timeframe.


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
ror_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:

                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)
                split_index = int(len(data) * 0.7)
                lastPrice = data['LastPrice'][split_index:]
                futurePrice = data['FuturePrice'][split_index:]
                # Calculate Rate of Return RoR
                ror = np.array((futurePrice - lastPrice)/lastPrice)
                # Calculate average RoR
                avg_ror = np.mean(ror)
                print(f"Average RoR for {img_type} {timeframe} {prediction}: {avg_ror}")
                # Add the evaluation metrics to the DataFrame
                ror_df = ror_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Average_RoR': avg_ror
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('ror.csv', index=False)

In [ ]:
ror_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:

            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)
            split_index = int(len(data) * 0.7)
            lastPrice = data['LastPrice'][split_index:]
            futurePrice = data['FuturePrice'][split_index:]
            # Calculate Rate of Return RoR
            ror = np.array((futurePrice - lastPrice)/lastPrice)
            # Calculate average RoR
            avg_ror = np.mean(ror)
            print(f"Average RoR for {timeframe} {prediction}: {avg_ror}")
            # Add the evaluation metrics to the DataFrame
            ror_df = ror_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Average_RoR': avg_ror
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('ror.csv', index=False)